# 3.2 BLASTn
Use the blastn analysis to identify additional genes that could be added to the model.

In [1]:
import cobra
import pandas as pd
import numpy as np
from cobra.io import load_json_model
from glob import glob
from cobra.manipulation.modify import rename_genes

In [2]:
# Load the model and get a list of gene IDs already present in Nissle model
EcN_ID = 'CP022686.1'
EcN_model = load_json_model('../data/models/%s.json'%EcN_ID)

EcN_listGeneIDs=[]
for gene in EcN_model.genes:
    EcN_listGeneIDs.append(gene.id)

## 1. Comparison between BLASTp and BLASTn

In [3]:
# Import both the BLASTp and BLASTn orthology matrix
ortho_matrix=pd.read_csv('../tables/ortho_matrix.csv')
ortho_matrix=ortho_matrix.set_index('Unnamed: 0')

ortho_matrix_unan =pd.read_csv('../tables/ortho_matrix_unan.csv')
ortho_matrix_unan=ortho_matrix_unan.set_index('Unnamed: 0')

### 1.1 Remove not present in pangenome and present in EcN model

In [4]:
# Get the number of genes represented within the 55 models
pangenome = pd.read_csv('../tables/pangenome.csv', usecols=[1], skiprows=1, names=['genes'])['genes'].tolist()
geneIDs_matrix = pd.read_csv('../tables/geneIDs_matrix.csv')
geneIDs_matrix.set_index('Unnamed: 0', inplace=True)

# Only keep the geneIDs that are present in the pangenome
modelIDs_matrix = geneIDs_matrix[geneIDs_matrix.isin(pangenome)]

# Rename all zeros as NaN and drop all lines with NaN
for strain in ortho_matrix.columns:
    modelIDs_matrix[strain] = np.where(ortho_matrix[strain] == 0, np.nan, modelIDs_matrix[strain])
    
modelIDs_matrix.dropna(axis = 0, how = 'all', inplace = True)
print(len(modelIDs_matrix))
modelIDs_matrix.head()

1571


,CU651637,CP001855,CP002167,CP000468,CP000946,CP000819,CP001665,AM946981,CP001509,CP001396,...,CU928163,CP000243,CP001063,CP000036,CP000034,CP001383,AE014073,AE005674,CP000266,CP000038
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
CIW80_00015,LF82_0064,NRG857_07005,UM146_09930,NaN,EcolC_2243,ECB_01370,ECBD_2225,B21_01382,ECD_01370,NaN,...,ECUMN_1663,UTI89_C1637,SbBS512_E1635,SBO_1672,NaN,SFxv_2014,S1475,SF1797,SFV_1791,NaN
CIW80_00090,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CIW80_00130,LF82_2802,NRG857_07110,UM146_09825,NaN,EcolC_2219,ECB_01397,ECBD_2199,B21_01408,ECD_01397,NaN,...,ECUMN_1688,UTI89_C1659,NaN,NaN,SDY_1734,NaN,NaN,NaN,NaN,SSON_1697
CIW80_00135,LF82_2803,NRG857_07115,UM146_09820,NaN,EcolC_2218,ECB_01398,ECBD_2198,B21_01409,ECD_01398,NaN,...,ECUMN_1689,UTI89_C1660,NaN,NaN,SDY_1733,NaN,NaN,NaN,NaN,SSON_1696
CIW80_00140,LF82_2804,NRG857_07120,UM146_09815,NaN,EcolC_2217,ECB_01399,ECBD_2197,B21_01410,ECD_01399,NaN,...,ECUMN_1690,UTI89_C1661,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# Remove all rows that are already present in the EcN model and remove all rows that are not in the pangenome
ortho_matrix = ortho_matrix.loc[~ortho_matrix.index.isin(EcN_listGeneIDs)]
# ortho_matrix = ortho_matrix.loc[ortho_matrix.index.isin(modelIDs_matrix.index.values)]
ortho_matrix_unan = ortho_matrix_unan.loc[~ortho_matrix_unan.index.isin(EcN_listGeneIDs)]
# ortho_matrix_unan = ortho_matrix_unan.loc[ortho_matrix_unan.index.isin(modelIDs_matrix.index.values)]

### Find the difference between the two matrixes

In [6]:
nuc_diff = ortho_matrix_unan - ortho_matrix
nuc_diff = nuc_diff.loc[~(nuc_diff==0).all(axis=1)]
nuc_diff['Blast difference'] = nuc_diff.sum(axis=1)
# nuc_diff = nuc_diff.loc[nuc_diff['Blast difference'] > 10] # Where do you see the biggest difference?
nuc_diff

,CU651637,CP001855,CP002167,CP000468,CP000946,CP000819,CP001665,AM946981,CP001509,CP001396,...,CP000243,CP001063,CP000036,CP000034,CP001383,AE014073,AE005674,CP000266,CP000038,Blast difference
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
CIW80_00020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,6.0
CIW80_00030,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
CIW80_00035,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0
CIW80_00040,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
CIW80_00045,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CIW80_25765,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15.0
CIW80_25780,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,2.0
CIW80_25785,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [7]:
# model_presence('ECOLIN_03670') > kdpF, Kabcpp, already present
# model_presence('ECOLIN_05255') > agaZ, TGBPA, already present
# model_presence('ECOLIN_26100') > aceB, MALS, already present

There are 174 genes that could potentially be added. However, as the three genes above show, several of these genes could be linked to reactions that are already present in the model. Use the same method as in notebook 3.1 and check for every reaction the presence in the model.

# Presence of reactions in EcN model

In [7]:
from cobra import Model, Reaction, Metabolite
from cobra.manipulation.modify import rename_genes

In [8]:
# From the ortho_matrix, create a list of genes for the CurrentStrain that have homology with EcN genes 
def ortho_to_list(CurrentStrain, geneIDs_matrix):

    # Load orthology matrix and drop all similarities with existing EcN model
    ortho_matrix = pd.read_csv('../tables/ortho_matrix_unan.csv') # use the BLASTn ortho_matrix instead of BLASTp matrix
    ortho_matrix.set_index('Unnamed: 0', inplace=True)

    # Rename the current strain in the geneIDs_matrix
    current_geneIDs_matrix = geneIDs_matrix[CurrentStrain].rename('geneID')

    # Merge ortho_matrix with current geneIDs matrix
    ortho_matrix = pd.merge(ortho_matrix, current_geneIDs_matrix, left_index=True, right_index=True)
    ortho_matrix.set_index('geneID', inplace=True)
    try: ortho_matrix.drop('None', inplace=True)
    except: pass

    # Create list of all genes that are not yet in model and have homology with current strain
    ortho_list = ortho_matrix[ortho_matrix[CurrentStrain] == 1.0].index.tolist()
    return(ortho_list)

In [9]:
# Based on the gene IDs in the ortho_list, retrieve reactions from the current_model and find out if they are not yet present in the EcN model
def gene_addition(modelCopy, current_model, ortho_list, geneIDs_matrix, CurrentStrain):
    add_gene = []
    
    # Dataframe to store the reactions that could be of interest
    genes_df = pd.DataFrame(columns=['reaction', 'EcN'])
    
    # for all genes that have homology, check if they should be added
    for gene in ortho_list:
        try:
            reactions = current_model.genes.get_by_id(gene).reactions
            for reaction in reactions:
                
                # check if the gene is already present in the model
                try:
                    modelCopy.reactions.get_by_id(reaction.id)
                
                # If the reaction is not yet in the model, check the dependencies of genes linked to the reaction
                # If the model contains an "and" statement, check if all involved genes are present in the ortho_list
                # If they are not all present, do not add the reaction
                except: 
                    grr = current_model.reactions.get_by_id(reaction.id).gene_reaction_rule # get gene reaction rule
                    or_statement = grr.split(' or ') #split based on "or" statement
                    for or_genes in or_statement:
                        if len(or_genes.split(' and ')) == 1: #check whether there is an "and" statement
                            add_gene.append(True) #if not there are only "or" statements, and having one of them in the ortho_list is sufficient
                        else:
                            and_present = True
                            involved_genes = or_genes.split(' and ') #create a list of all the genes in the "and" statement
                            for gene_x in involved_genes:
                                if gene_x.lstrip('(').rstrip(')') not in ortho_list: #check for each gene if it's present in the ortho_list
                                    and_present = False #if one of the genes is not present, do not give True for addition on this part
                            if and_present == True:
                                add_gene.append(True)
                                
                    if True in add_gene: #only if one of the "or" statements (one gene or combination of "and" genes) is completely present, add reaction
                        # modelCopy.add_reactions([current_model.reactions.get_by_id(reaction.id)]) # if not present, add
                        genes_df.loc[gene, 'reaction'] = reaction.id
                        genes_df.loc[gene, 'EcN'] = geneIDs_matrix[geneIDs_matrix[CurrentStrain]==gene].index.values
                        add_gene = []
                    else:
                        continue
#                          print('Not added:', reaction.id)
        except:
            continue
    return(genes_df)

In [10]:
# Load the EcN model
EcN_ID = 'CP022686.1'
EcN_model = load_json_model('../data/models/%s.json'%EcN_ID)
modelCopy = EcN_model.copy()

# Load strain info to get the model for each strain
strain_info = pd.read_csv('../tables/strain_info.csv', usecols=[1,3,6])

#load the orthoID matrix. Set index, rename EcN and remove all genes == None
orthoIDs_matrix = pd.read_csv('../tables/orthoIDs_matrix.csv')
orthoIDs_matrix = orthoIDs_matrix.set_index('Unnamed: 0') #Set EcN gene IDs as index
orthoIDs_matrix = orthoIDs_matrix[orthoIDs_matrix.index != 'None'] # Remove all indexes 00 'None'

orthoIDs_matrix.head()

,CU651637,CP001855,CP002167,CP000468,CP000946,CP000819,CP001665,AM946981,CP001509,CP001396,...,CU928163,CP000243,CP001063,CP000036,CP000034,CP001383,AE014073,AE005674,CP000266,CP000038
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
CIW80_00005,LF82_1032,NRG857_06995,UM146_09940,APECO1_564,EcolC_2245,ECB_01368,ECBD_2227,B21_01380,ECD_01368,BWG_1240,...,ECUMN_1659,UTI89_C1635,SbBS512_E1633,SBO_1674,SDY_1774,SFxv_2017,S1472,SF1802,SFV_1794,SSON_1733
CIW80_00010,LF82_2792,NRG857_07000,UM146_09935,APECO1_565,EcolC_2244,ECB_01369,ECBD_2226,B21_01381,ECD_01369,BWG_1241,...,ECUMN_1662,UTI89_C1636,SbBS512_E1634,SBO_1673,None,SFxv_2015,None,None,SFV_1792,SSON_1732
CIW80_00015,LF82_0064,NRG857_07005,UM146_09930,APECO1_566,EcolC_2243,ECB_01370,ECBD_2225,B21_01382,ECD_01370,BWG_1242,...,ECUMN_1663,UTI89_C1637,SbBS512_E1635,SBO_1672,None,SFxv_2014,S1475,SF1797,SFV_1791,None
CIW80_00020,LF82_0802,NRG857_07010,UM146_09925,APECO1_567,EcolC_2242,ECB_01371,ECBD_2224,B21_01383,ECD_01371,None,...,ECUMN_1664,UTI89_C1638,SbBS512_E1636,SBO_1671,None,None,None,SF1795,SFV_1790,SSON_1725
CIW80_00025,LF82_0397,NRG857_07015,UM146_09920,APECO1_568,EcolC_2241,ECB_01374,ECBD_2221,B21_01386,ECD_01374,BWG_1243,...,ECUMN_1665,UTI89_C1639,SbBS512_E1642,SBO_1669,SDY_1764,SFxv_2012,S1478,SF1794,SFV_1789,SSON_1724


In [11]:
# Loop over the strains and find out which reactions are not yet present in the EcN model

all_genes_df =  pd.DataFrame(columns=['reaction', 'EcN'])

for strain in strain_info['NCBI ID']:
    CurrentStrain = strain

    # Load model
    CurrentModel = strain_info.loc[strain_info['NCBI ID'] == CurrentStrain, 'Model Name'].values[0]
    current_model = cobra.io.load_json_model('../data/models/%s.json'%CurrentModel)
    print("Checking reactions from strain: ", CurrentStrain,', model:', current_model.id)
    
    # Get ortho_list without reactions already present in model
    ortho_list = ortho_to_list(CurrentStrain, orthoIDs_matrix)
    
    # Add reactions of each model to the EcN model
    genes_df = gene_addition(modelCopy, current_model, ortho_list, geneIDs_matrix, CurrentStrain)
    all_genes_df = all_genes_df.append(genes_df, ignore_index=True)

Checking reactions from strain:  CU651637 , model: iLF82_1304
Checking reactions from strain:  CP001855 , model: iNRG857_1313
Checking reactions from strain:  CP002167 , model: iUMN146_1321
Checking reactions from strain:  CP000468 , model: iAPECO1_1312
Checking reactions from strain:  CP000946 , model: iEcolC_1368
Checking reactions from strain:  CP000819 , model: iECB_1328
Checking reactions from strain:  CP001665 , model: iECBD_1354
Checking reactions from strain:  AM946981 , model: iB21_1397
Checking reactions from strain:  CP001509 , model: iECD_1391
Checking reactions from strain:  CP001396 , model: iBWG_1329
Checking reactions from strain:  CP001637 , model: iEcDH1_1363
Checking reactions from strain:  AP012030 , model: iECDH1ME8569_1439
Checking reactions from strain:  CU928162 , model: iECED1_1282
Checking reactions from strain:  CP000802 , model: iEcHS_1320
Checking reactions from strain:  CU928160 , model: iECIAI1_1343
Checking reactions from strain:  CP002516 , model: iEKO1

In [12]:
# inspect the dataframe
all_genes_df.head()

,reaction,EcN


There are no reactions to be added that we not yet added in the construction of the draft model